# Solvers' strategy

The goal is to test some strategy, check what happen, what works. Performance of the implementation is not a priority. In this first step I want to find different strategies and evaluate what works.

This step will also help to define what features are requied in the board and game implementation.

My goal is to show how to analyse a problem. I know that there are solvers for this problem (the nonogram library for instance). And it may be a funny topics for mathematicians. 

In the real lie I recommend you check first for some known algorithm or strategy. But you may not be lucky and the first step will be to analyse, breal the problem in solvable pieces until you find a known algorithm you can apply to a part of the problem.



# Mimic manual process

This strategy mimics what we done manually to solve the board on a Nonogram game.

You may implement a series of rules, For instance:
- if the number of blacks for a row is the width of the board  then all cells of the row are black 
- a similare situation is when the number of blacks + the number of blocks -1 is the width of the board (think for yourself thzat there is onky one option). The row alternate black cells by the clues and fillers of size one.
- obviusly same apply for cols
- and also if a black is on the edge then I yiu can place all the cells on the contiguous cells block
...

Well it is tedious, tricky to implement. And not funny!

I will not implement it. However trying to solve the problem help understanding the internals of the problem and what works in AI.

For instance, when a human solve the board, doing tons of computations for each cell is boring. Human tend to rely on more visual clues like proximity, overall shape and symetry. Most of this information is not provided by the clues but our human brain knows that the puzzle boils down to an image and there must be some logic in this image. Nonogram are not supposed to be random scattered plots. This reasoning is far beyond what the numeric algorithm can do. ON the other hand computations for a 15x15 matrix are cheap and fast. 


Putting some rules in the AI may be more efficient than bare computation if rules are carefully chosen.

# General resolution patterns

# What problem I am solving ?

It seems obvious that the goal is to place blacks on a board. But think for ourself for a 1 minute what is going on if yu limit yourself to this one dimension.

When the nonogram becomes harder the number of black clues go down. The you are in a situation where there are a ot of way to place this blacks. 

You may find more usefull to place fillers. Say the clue is 2 blacks over a row of 15. Once you have place at least 1 black you may assume that cells on each side may be black or filler, and can you can place fillers for 12 cells before and after with confidence.

Thus the strategy becomes: find where the black are, and also where the fillers are. 

The basic steps for this strategy will be to fill a board with 0 and 1 with respect to the clues. 

Different patterns might be used
- brute force: compute all combinations and filter the valid ones
- dynamic programming: build up the board by solving subboards 
- path optimization:
- genetic
- constraint programming

Now let us take a new step. What if they were something between black or filler, true or false. Probabilty of being black.

Intuitively the intersection between rows and cols clues with large value is more likely to be black. And conversely intersection of rows and cells with low value is likely to be a filler. 

I can compute the probability for the cell to be black from the clue and refine the probability while other cells are validated to be either black or filler. Note that clues are used for initialisation but I do not use the clues later to choose black or filler. It is black when the probability hits 1, and filler when it hits 0.

Does it work ? Well I do not know yet.

This is more or less what a Machine Learning algorithm would do. It would learn probabilities of being black from examples. 

Spaaking about Machine Learning, how would I put the problem ?

Idea 1: 
- take as input the clues translated into a series of features or a text
- output the probabilities of black for each board position
- put some hidden layers in between to learn how to transform clues into board
- then train it on a bunch of solved puzzles

Stated as text it looks like some NLP algorithm extracting concepts from a text.

There are a few problems here
- representing the clues into features moght be tricky. Depending on the way the blocks are placed there might be different number of features. Feature 4 may also represent row 2, or 3 or 4 depending on how the cells are grouped in rows.
- I need to solve a large number of puzzles with another technique to get the labelled data (the solved bpard) the machine will learn from
- Not sure it converges as all board will be different

May be I can train a more simple problem. What do I need to build a nono robot ? Usually the game get each action of marking a cell black or filler one after the other and will confirm or invalidate the action.

What if I train a robot to tell whether a cell is black or filler (or the probability of being black) given the clues for row and col and  the current state of the row and cell where the cell intersects ?

This robot could learn from a trial and error process with Reinforcement Learning algorithms.

- There is a much simplier problem
- The input has still more variation but I can put a maximum number of blocks and set unused values to 0. It still make sense as it still represent the same row. The text input is also an option
- I can get a lot of sample from one board to pretrain a model
- It can learn more easily from samples close from each other 
- It is easier to check the labels and inference because you just have to solve a cell 


A more simple way to use Machine Learning in that case is to use some genetic algorithm process. 

Say I do something as below
- produce 100 random boards
- keep the 10 that placed the most blacks
- produce 9 copies of each 10 and randomly alter 25% of the cells
- produce 10 random board
- loop until a board is done

Does it converges to a solution ? "rien n'est moins sur"

First it learn how to solve a given nonogram, it does not learn rules to solve a nonogram

How may I learn the most valid rules to solve a nonogram ?

When trying to find a way through this, I feature out that a list of actions may be drawing lines staring at some position and being a certain numbers of cells (the clue number for a row/col or a split of row/col).

I find the representation interesting because 
- instead of CNxCM clues -> a BNxBM board
- I have CNxCM clues -> CNxCM indices (start row and col for each clue)

The problem space is smaller and grows less faster with size.

The genertic algorithm will refine a sort of script made of a list of indices (one for each bar of cells)
- produce 100 random scripts. this board is leass random because the overall number of bars is correct but they are not at the right place
- keep the 10 that placed the most blacks
- produce 9 copies of each 10 and randomly alter 25% of the position by moving up or down
- produce 10 random board
- loop until a board is done

I want to test this with the same patterns than "putting 0 or 1" on the board

# Put 0 or 1 on each cell of the board 

# Requirements

Clues management features 
- count the total number of blacks per row/col and at the board level
- check clues for consistency (same number of blacks in rows and cols clues)

Game rules
- check whether the board holds the rules given by the clue
- evaluate how much I am close frol the objective (for instance, how many blacks are placed correctly)

Board 
- allow to solve a board with hints (dynamic programming)